# Title

In [1]:
from utils.test_file import generate_group_sequence
from algorithms.online import FirstFit, BestFit, WorstFit, MinCovidChairs, Hybrid_BF_CC
import pandas as pd
from online_batch import run_algorithm_with_original_groups, repeat_algorithm_with_different_groups, get_file_names

## Set variables

We need to define:

* which files (=grids) we are going to run
* how many random groups we gonna test per grids
* the number of groups
* which algorithms

In [2]:
FILE_DIR = "input/online"
# 100 different group sequences from each 50 groups long
GROUPS_LIST = [generate_group_sequence(50) for i in range(10)]
ALGORITHMS = [FirstFit, BestFit, WorstFit, MinCovidChairs, Hybrid_BF_CC]

Gather all the files that we want to run on.

In [3]:
file_names = get_file_names(FILE_DIR)

Run the algorithms and gather the results in a dataframe

In [4]:
%%capture
result = {}
for file in file_names:
    grid_path = f"{FILE_DIR}/{file}"

    grid_result = {}
    for algorithm in ALGORITHMS:
        alg_filled_chairs = []
        alg_filled_chairs.append(run_algorithm_with_original_groups(algorithm, grid_path))
        alg_filled_chairs = alg_filled_chairs + repeat_algorithm_with_different_groups(algorithm, grid_path, GROUPS_LIST)
        alg_name = str(algorithm.__name__)
        grid_result.update({alg_name: alg_filled_chairs})
    
    result.update({file: grid_result})


In [5]:
print("The following grids were solved:")    
print(result.keys())

The following grids were solved:
dict_keys([&#39;Online1.txt&#39;, &#39;Online10.txt&#39;, &#39;Online11.txt&#39;, &#39;Online12.txt&#39;, &#39;Online2.txt&#39;, &#39;Online3.txt&#39;, &#39;Online4.txt&#39;, &#39;Online5.txt&#39;, &#39;Online6.txt&#39;, &#39;Online7.txt&#39;, &#39;Online8.txt&#39;, &#39;Online9.txt&#39;])


Put everything in a dataframe:

In [6]:
df_list = []
for grid, algs in result.items():
    alg_series = []
    for alg, value_list in algs.items():
        alg_series.append(pd.Series(value_list, name=alg))
    df = pd.DataFrame(alg_series).transpose()
    df = df.assign(grid=grid).set_index('grid', append=True, drop=True)
    df_list.append(df)
df = pd.concat(df_list)

Let's extract the records with the original groups from the input grid files, and put it in a seperate dataframe

In [7]:
df_real_groups = df.xs(0, level=None)
df_real_groups

,FirstFit,BestFit,WorstFit,MinCovidChairs,Hybrid_BF_CC
grid,,,,,
Online1.txt,5,5,5,5,5
Online10.txt,74,67,73,79,67
Online11.txt,45,52,39,52,52
Online12.txt,144,147,120,141,146
Online2.txt,9,9,5,5,9
Online3.txt,15,11,13,15,15
Online4.txt,12,12,12,16,16
Online5.txt,22,26,22,22,26
Online6.txt,29,29,27,33,34


We also make a separate dataframe for the generated group simulation records: 

In [8]:
df_sim = df.loc[pd.IndexSlice[range(1,11),:]]
df_sim

FirstFit  BestFit  WorstFit  MinCovidChairs  Hybrid_BF_CC
   grid                                                                   
1  Online1.txt          6        6         6               6             6
   Online10.txt        65       65        67              75            65
   Online11.txt        56       60        46              61            61
   Online12.txt       132      139       117             137           141
   Online2.txt          9        9         7               7             9
...                   ...      ...       ...             ...           ...
10 Online5.txt         21       23        23              23            25
   Online6.txt         27       34        25              29            30
   Online7.txt         34       38        33              39            41
   Online8.txt         42       45        41              44            44
   Online9.txt         62       62        58              66            62

[120 rows x 5 columns]

The following analyses are on the simulation results.

In [9]:
df_sim.groupby(['grid']).mean()

,FirstFit,BestFit,WorstFit,MinCovidChairs,Hybrid_BF_CC
grid,,,,,
Online1.txt,5.2,5.4,4.5,5.4,5.4
Online10.txt,71.3,69.3,68.3,74.4,69.3
Online11.txt,57.2,59.3,45.8,61.1,61.1
Online12.txt,132.1,134.1,116.3,139.1,138.9
Online2.txt,7.8,7.9,6.9,7.3,7.9
Online3.txt,15.6,15.1,14.9,16.0,16.3
Online4.txt,24.9,25.6,22.0,25.0,26.3
Online5.txt,23.7,23.8,22.4,25.2,24.5
Online6.txt,30.7,32.2,26.1,32.0,31.8


In [11]:
df_sim.groupby(['grid']).std()

,FirstFit,BestFit,WorstFit,MinCovidChairs,Hybrid_BF_CC
grid,,,,,
Online1.txt,1.229273,1.074968,0.971825,1.074968,1.074968
Online10.txt,2.983287,4.083844,3.164034,2.170509,4.083844
Online11.txt,1.686548,4.029061,3.326660,2.643651,2.469818
Online12.txt,3.107339,4.012481,5.396501,3.541814,6.436873
Online2.txt,0.788811,0.994429,0.567646,0.483046,0.994429
Online3.txt,0.843274,1.663330,1.197219,1.054093,1.159502
Online4.txt,1.370320,1.776388,1.825742,1.247219,0.948683
Online5.txt,1.567021,2.898275,1.837873,1.873796,2.173067
Online6.txt,2.002776,2.201010,2.233582,2.108185,1.873796


In [12]:
df_sim.groupby(['grid']).max()

,FirstFit,BestFit,WorstFit,MinCovidChairs,Hybrid_BF_CC
grid,,,,,
Online1.txt,7,7,6,7,7
Online10.txt,76,76,72,80,76
Online11.txt,60,64,50,66,64
Online12.txt,138,139,123,144,150
Online2.txt,9,9,8,8,9
Online3.txt,17,18,17,18,18
Online4.txt,27,27,24,27,27
Online5.txt,27,28,25,29,28
Online6.txt,34,36,30,35,34
